In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import geopandas as gpd
import pandas as pd
import shapely
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
credentials = service_account.Credentials.from_service_account_file(
    'C:/Users/pfbab/NSS/googlekey/solar-roof-capstone-787ff1e58b5f.json',
)
metromap = gpd.read_file('../data/ratesbymetro/ratesbymetro.shp')

In [3]:
client = bigquery.Client(credentials=credentials)

In [4]:
query = """SELECT region_name,state_name, center_point, lat_max, lat_min, lng_max, lng_min 
FROM `bigquery-public-data.sunroof_solar.solar_potential_by_censustract` AS solar
WHERE yearly_sunlight_kwh_total > 0
    """

In [5]:
query_job = client.query(query)


In [6]:
solartract = query_job.to_dataframe()
solartract

,region_name,state_name,center_point,lat_max,lat_min,lng_max,lng_min
0,13115000201,Georgia,POINT(-85.1693420410156 34.3307456970215),34.371120,34.287239,-85.117592,-85.231651
1,18057110512,Indiana,POINT(-86.0604782104492 40.0928688049316),40.101791,40.076389,-86.038544,-86.090332
2,32031980200,Nevada,POINT(-119.929153442383 39.588264465332),39.635529,39.548290,-119.845078,-120.001953
3,01055010700,Alabama,POINT(-85.8930816650391 34.0650825500488),34.152290,33.987690,-85.822540,-85.987312
4,42043024700,Pennsylvania,POINT(-76.8120880126953 40.4255256652832),40.571880,40.336391,-76.535408,-77.027008
...,...,...,...,...,...,...,...
52564,26065004491,Michigan,POINT(-84.4735107421875 42.7307472229004),42.732880,42.728729,-84.472054,-84.475311
52565,48041002015,Texas,POINT(-96.3363723754883 30.6165580749512),30.627750,30.604610,-96.323433,-96.348572
52566,12011100104,Florida,POINT(-80.1211624145508 25.9805011749268),25.986570,25.975019,-80.119270,-80.123306
52567,51107611020,Virginia,POINT(-77.4626388549805 39.0011787414551),39.024670,38.978981,-77.428917,-77.506088


In [7]:
solartract = solartract.astype({'lat_max':'float','lat_min':'float','lng_max':'float','lng_min':'float'})
def solarshapes(row):
    shape = shapely.geometry.Polygon([(row['lng_min'], row['lat_min']), (row['lng_max'], row['lat_max']), (row['lng_max'], row['lat_min']),(row['lng_min'], row['lat_max'])])
    return shape

In [8]:
shapes = []
for index, row in solartract.iterrows():
    shapes.append(solarshapes(row))

In [9]:
solarshapes = solartract.drop(columns='center_point')
solarshapes['shapes'] = shapes
solarshapes = gpd.GeoDataFrame(solarshapes, crs=metromap.crs, geometry='shapes')

In [10]:
solartract['center_point'] = gpd.GeoSeries.from_wkt(solartract['center_point'])


In [11]:
solartract = gpd.GeoDataFrame(solartract, crs=metromap.crs, geometry='center_point')

In [12]:
solartract

,region_name,state_name,center_point,lat_max,lat_min,lng_max,lng_min
0,13115000201,Georgia,POINT (-85.16934 34.33075),34.371120,34.287239,-85.117592,-85.231651
1,18057110512,Indiana,POINT (-86.06048 40.09287),40.101791,40.076389,-86.038544,-86.090332
2,32031980200,Nevada,POINT (-119.92915 39.58826),39.635529,39.548290,-119.845078,-120.001953
3,01055010700,Alabama,POINT (-85.89308 34.06508),34.152290,33.987690,-85.822540,-85.987312
4,42043024700,Pennsylvania,POINT (-76.81209 40.42553),40.571880,40.336391,-76.535408,-77.027008
...,...,...,...,...,...,...,...
52564,26065004491,Michigan,POINT (-84.47351 42.73075),42.732880,42.728729,-84.472054,-84.475311
52565,48041002015,Texas,POINT (-96.33637 30.61656),30.627750,30.604610,-96.323433,-96.348572
52566,12011100104,Florida,POINT (-80.12116 25.98050),25.986570,25.975019,-80.119270,-80.123306
52567,51107611020,Virginia,POINT (-77.46264 39.00118),39.024670,38.978981,-77.428917,-77.506088


In [13]:
tractmerge = gpd.sjoin(metromap.loc[:,['GEOID10','geometry']], solartract, how='right', op='intersects')
tractmerge

,index_left,GEOID10,region_name,state_name,center_point,lat_max,lat_min,lng_max,lng_min
0,292.0,76204,13115000201,Georgia,POINT (-85.16934 34.33075),34.371120,34.287239,-85.117592,-85.231651
1,1502.0,41212,18057110512,Indiana,POINT (-86.06048 40.09287),40.101791,40.076389,-86.038544,-86.090332
2,NaN,NaN,32031980200,Nevada,POINT (-119.92915 39.58826),39.635529,39.548290,-119.845078,-120.001953
3,NaN,NaN,01055010700,Alabama,POINT (-85.89308 34.06508),34.152290,33.987690,-85.822540,-85.987312
4,NaN,NaN,42043024700,Pennsylvania,POINT (-76.81209 40.42553),40.571880,40.336391,-76.535408,-77.027008
...,...,...,...,...,...,...,...,...,...
52564,2316.0,47719,26065004491,Michigan,POINT (-84.47351 42.73075),42.732880,42.728729,-84.472054,-84.475311
52565,2491.0,18748,48041002015,Texas,POINT (-96.33637 30.61656),30.627750,30.604610,-96.323433,-96.348572
52566,4.0,56602,12011100104,Florida,POINT (-80.12116 25.98050),25.986570,25.975019,-80.119270,-80.123306
52567,1976.0,92242,51107611020,Virginia,POINT (-77.46264 39.00118),39.024670,38.978981,-77.428917,-77.506088


In [14]:
tractmerge[tractmerge['GEOID10'].isna()]

,index_left,GEOID10,region_name,state_name,center_point,lat_max,lat_min,lng_max,lng_min
2,NaN,NaN,32031980200,Nevada,POINT (-119.92915 39.58826),39.635529,39.548290,-119.845078,-120.001953
3,NaN,NaN,01055010700,Alabama,POINT (-85.89308 34.06508),34.152290,33.987690,-85.822540,-85.987312
4,NaN,NaN,42043024700,Pennsylvania,POINT (-76.81209 40.42553),40.571880,40.336391,-76.535408,-77.027008
5,NaN,NaN,47073050700,Tennessee,POINT (-82.76249 36.44636),36.532211,36.363400,-82.610168,-82.933983
6,NaN,NaN,51021040200,Virginia,POINT (-81.24263 37.11118),37.275311,36.952419,-81.091560,-81.462212
...,...,...,...,...,...,...,...,...,...
52536,NaN,NaN,06001425101,California,POINT (-122.30653 37.84037),37.848110,37.832352,-122.294327,-122.320251
52543,NaN,NaN,37051980200,North Carolina,POINT (-78.93377 35.16167),35.202919,35.118690,-78.901901,-78.972031
52548,NaN,NaN,06073021302,California,POINT (-116.77920 32.66676),32.795620,32.557781,-116.668152,-116.938377
52562,NaN,NaN,37063002100,North Carolina,POINT (-78.87281 36.17469),36.239540,36.071381,-78.802002,-78.966789


In [15]:
center = metromap[metromap.GEOID10 == '30898'].geometry.centroid
area_center = [center.y, center.x]
print(area_center)

[3600    37.829925
dtype: float64, 3600   -94.703396
dtype: float64]


<ipython-input-15-f7f33222da28>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = metromap[metromap.GEOID10 == '30898'].geometry.centroid


In [16]:
solarpoints = folium.Map(location = area_center, zoom_start = 4)

marker_cluster = MarkerCluster().add_to(solarpoints)

#folium.GeoJson(metromap).add_to(testmap)

# folium.Choropleth(
#     geo_data=metromap,
#     data=metromap,
#     columns=['GEOID10','ALAND10'],
#     key_on='feature.properties.GEOID10',
#     fill_color='YlOrRd',
#     fill_opacity=0.7,
#     line_opacity=0.5,
#     bins=8
#              ).add_to(testmap)

for row_i, row_val in tractmerge[tractmerge['GEOID10'].isna()].iterrows():
    center = row_val.center_point
    loc = [center.y,center.x]
    pop = str(row_val['region_name']) + str(" - ") + str(row_val['state_name']) + str(' - ') + str(row_val['center_point']) 
    if row_val['lat_max'] > 0:
        icon = folium.Icon(icon='hand-scissors-o', prefix='fa', color='green')
    else:
        icon = folium.Icon(icon='hand-scissors-o', prefix='fa', color='red')
    
    marker = folium.Marker(
        location = loc,
        popup = pop,
        icon = icon)
    marker.add_to(marker_cluster)
    
solarpoints.save('../data/solarpoints.html')

In [17]:
tractmerge

,index_left,GEOID10,region_name,state_name,center_point,lat_max,lat_min,lng_max,lng_min
0,292.0,76204,13115000201,Georgia,POINT (-85.16934 34.33075),34.371120,34.287239,-85.117592,-85.231651
1,1502.0,41212,18057110512,Indiana,POINT (-86.06048 40.09287),40.101791,40.076389,-86.038544,-86.090332
2,NaN,NaN,32031980200,Nevada,POINT (-119.92915 39.58826),39.635529,39.548290,-119.845078,-120.001953
3,NaN,NaN,01055010700,Alabama,POINT (-85.89308 34.06508),34.152290,33.987690,-85.822540,-85.987312
4,NaN,NaN,42043024700,Pennsylvania,POINT (-76.81209 40.42553),40.571880,40.336391,-76.535408,-77.027008
...,...,...,...,...,...,...,...,...,...
52564,2316.0,47719,26065004491,Michigan,POINT (-84.47351 42.73075),42.732880,42.728729,-84.472054,-84.475311
52565,2491.0,18748,48041002015,Texas,POINT (-96.33637 30.61656),30.627750,30.604610,-96.323433,-96.348572
52566,4.0,56602,12011100104,Florida,POINT (-80.12116 25.98050),25.986570,25.975019,-80.119270,-80.123306
52567,1976.0,92242,51107611020,Virginia,POINT (-77.46264 39.00118),39.024670,38.978981,-77.428917,-77.506088


In [18]:
shapemerge = gpd.sjoin(metromap.loc[:,['GEOID10','geometry']], solarshapes, how='right', op='intersects')
shapemerge

,index_left,GEOID10,region_name,state_name,lat_max,lat_min,lng_max,lng_min,shapes
0,292.0,76204,13115000201,Georgia,34.371120,34.287239,-85.117592,-85.231651,"POLYGON ((-85.23165 34.28724, -85.11759 34.371..."
1,1502.0,41212,18057110512,Indiana,40.101791,40.076389,-86.038544,-86.090332,"POLYGON ((-86.09033 40.07639, -86.03854 40.101..."
2,1880.0,74179,32031980200,Nevada,39.635529,39.548290,-119.845078,-120.001953,"POLYGON ((-120.00195 39.54829, -119.84508 39.6..."
3,616.0,32113,01055010700,Alabama,34.152290,33.987690,-85.822540,-85.987312,"POLYGON ((-85.98731 33.98769, -85.82254 34.152..."
4,916.0,37081,42043024700,Pennsylvania,40.571880,40.336391,-76.535408,-77.027008,"POLYGON ((-77.02701 40.33639, -76.53541 40.571..."
...,...,...,...,...,...,...,...,...,...
52564,2316.0,47719,26065004491,Michigan,42.732880,42.728729,-84.472054,-84.475311,"POLYGON ((-84.47531 42.72873, -84.47205 42.732..."
52565,2491.0,18748,48041002015,Texas,30.627750,30.604610,-96.323433,-96.348572,"POLYGON ((-96.34857 30.60461, -96.32343 30.627..."
52566,4.0,56602,12011100104,Florida,25.986570,25.975019,-80.119270,-80.123306,"POLYGON ((-80.12331 25.97502, -80.11927 25.986..."
52567,1976.0,92242,51107611020,Virginia,39.024670,38.978981,-77.428917,-77.506088,"POLYGON ((-77.50609 38.97898, -77.42892 39.024..."


In [19]:
shapemerge[shapemerge['GEOID10'].isna()]

,index_left,GEOID10,region_name,state_name,lat_max,lat_min,lng_max,lng_min,shapes
19,NaN,NaN,19103010200,Iowa,41.862091,41.780491,-91.555763,-91.831589,"POLYGON ((-91.83159 41.78049, -91.55576 41.862..."
84,NaN,NaN,42073010400,Pennsylvania,41.126499,41.061508,-80.383919,-80.519188,"POLYGON ((-80.51919 41.06151, -80.38392 41.126..."
153,NaN,NaN,25005646104,Massachusetts,41.581940,41.480331,-71.021187,-71.126862,"POLYGON ((-71.12686 41.48033, -71.02119 41.581..."
158,NaN,NaN,41003010400,Oregon,44.537491,44.280849,-123.183983,-123.501663,"POLYGON ((-123.50166 44.28085, -123.18398 44.5..."
214,NaN,NaN,48181001803,Texas,33.502380,33.399479,-96.524872,-96.656609,"POLYGON ((-96.65661 33.39948, -96.52487 33.502..."
...,...,...,...,...,...,...,...,...,...
52122,NaN,NaN,51011040200,Virginia,37.552540,37.340721,-78.705009,-79.018532,"POLYGON ((-79.01853 37.34072, -78.70501 37.552..."
52189,NaN,NaN,35049980000,New Mexico,35.572590,35.558899,-106.053619,-106.076309,"POLYGON ((-106.07631 35.55890, -106.05362 35.5..."
52207,NaN,NaN,13179010101,Georgia,32.058460,31.866261,-81.404732,-81.671371,"POLYGON ((-81.67137 31.86626, -81.40473 32.058..."
52221,NaN,NaN,12009980000,Florida,28.569170,28.409220,-80.520882,-80.640442,"POLYGON ((-80.64044 28.40922, -80.52088 28.569..."


In [20]:
solarshapes = folium.Map(location = area_center, zoom_start = 4)

marker_cluster = MarkerCluster().add_to(solarshapes)

#folium.GeoJson(metromap).add_to(testmap)

# folium.Choropleth(
#     geo_data=metromap,
#     data=metromap,
#     columns=['GEOID10','ALAND10'],
#     key_on='feature.properties.GEOID10',
#     fill_color='YlOrRd',
#     fill_opacity=0.7,
#     line_opacity=0.5,
#     bins=8
#              ).add_to(testmap)

for row_i, row_val in shapemerge[shapemerge['GEOID10'].isna()].iterrows():
    #center = row_val.shapes.geometry.centroid
    loc = [(row_val.lat_max + row_val.lat_min)/2,(row_val.lng_max + row_val.lng_min)/2]
    pop = str(row_val['region_name']) + str(" - ") + str(row_val['state_name']) + str(' - ') + str(row_val['shapes']) 
    if row_val['lat_max'] > 0:
        icon = folium.Icon(icon='hand-scissors-o', prefix='fa', color='green')
    else:
        icon = folium.Icon(icon='hand-scissors-o', prefix='fa', color='red')
    
    marker = folium.Marker(
        location = loc,
        popup = pop,
        icon = icon)
    marker.add_to(marker_cluster)
    
solarshapes.save('../data/solarshapes.html')

In [21]:
georegions = shapemerge.drop(columns=['shapes','index_left','lat_max','lat_min','lng_max','lng_min'])

In [22]:
georegions.to_csv('../data/georegions.csv', index=False)